In [5]:
import cv2
import numpy as np

In [6]:
def detect_color(hue):
    # Determine the color based on hue
    if 45/2 <= hue < 75/2:  # Yellow
        return f"Yellow{hue}"
    elif 105/2 <= hue < 135/2:  # Green
        return f"Green{hue}"
    elif 225/2 <= hue < 255/2:  # Blue
        return f"Blue{hue}"
    else:  # Red
        return f"Red{hue}"


In [7]:
def shape_and_color(frame):
    # removing noise
    blur = cv2.GaussianBlur(frame, (3, 3), 0)

    # removing points can exist on the image
    blur = cv2.medianBlur(blur, 5)
    
    edges = cv2.Canny(blur, 40 , 80)

    # morphological operation if the edge is not complete
    kernel = np.ones((3,3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=2)

    cv2.imshow("Edges", edges)

    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    shape:str

    if(len(contours) == 0 ): 
        shape = "None"
    else:
        contour = max(contours, key=cv2.contourArea)
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)


        hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)  # Convert the image to HSV
        mask = np.zeros(hsv_image.shape[:-1] , dtype=np.uint8)  # Create a new mask for each shape
        # print("mask shape: ", mask.shape , "hsv_image shape: ", hsv_image.shape)
        cv2.drawContours(mask, [contour], -1, 255, -1)  # Fill the shape on the mask
        h,*_ = cv2.mean(hsv_image, mask=mask)  # Get the mean color
        color = detect_color(h)
        

        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            shape = "Square"
        elif len(approx) > 4:
            shape = "Circle"
        else:
            shape = "Unkown"

        # Draw the shape on the image
        cv2.putText(frame, shape+color , (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Shapes", frame)


In [8]:
cap = cv2.VideoCapture(0)
cap.open("http://192.168.43.1:8080/video")

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    # frame = cv2.rotate( frame, cv2.ROTATE_90_CLOCKWISE)
    frame = cv2.resize(frame, (640, 480))
    
    shape_and_color(frame)

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1) & 0xFF	
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()